<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/KISTA_AUTOMATIC1111_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Kista AUTOMATIC1111 WebUI - v1.2.0

#@markdown This Colab is maintained by [github.com/steinhaug/stable-diffusion](https://github.com/steinhaug/stable-diffusion/) and lets you load Automatic1111 and Stable Diffusion with a Colab notebook. Play this cell to verify if you are running with a GPU and how much RAM.

#@markdown <a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/SD_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [6]:
#@title Load and verify settings
#@markdown ![Load your config](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/colab-split-load.png)

from pathlib import Path
from IPython.display import display, Markdown, clear_output
from ipywidgets import widgets, GridspecLayout, Layout
from configparser import ConfigParser

inip = Path("/content/drive/MyDrive")
if not inip.is_dir():
    from google.colab import drive
    drive.mount('/content/drive')
    clear_output()
else:
    print("Drive already mounted, continue.");
    clear_output()

INI_FILE_LOC = "/content/drive/MyDrive/sd-colab.ini"

def strtobool (val):
    """Quick cast tro boolean
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1'):
        return True
    elif val in ('', 'n', 'no', 'f', 'false', 'off', '0'):
        return False
    else:
        raise ValueError("invalid truth value %r" % (val,))

def getValue (savedconf, key, type):
    try:
        val = savedconf[key]
    except KeyError:
        val = ""
    if type == 'bool':
        return strtobool(val)
    else:
        return val

ext_controlnet = widgets.Checkbox( value=False, description='Controlnet 1.1', disabled=False, indent=False )
ext_openposeeditor = widgets.Checkbox( value=False, description='Open Pose Editor', disabled=False, indent=False )
ext_civitai = widgets.Checkbox( value=False, description='Image and civitai', disabled=False, indent=False )
ext_huggingface = widgets.Checkbox( value=False, description='Huggingface weui', disabled=False, indent=False )
ext_webuimodeltoolkit = widgets.Checkbox( value=False, description='Model Toolkit', disabled=False, indent=False )
ext_ultimateupscale = widgets.Checkbox( value=False, description='Ultimate Upscale', disabled=False, indent=False )
ext_additionalnetworks = widgets.Checkbox( value=False, description='Additional Networks', disabled=False, indent=False )
ext_dynamicprompts = widgets.Checkbox( value=False, description='Dynamic Prompts', disabled=False, indent=False )
ext_datasettageditor = widgets.Checkbox( value=False, description='Dataset Tag Editor', disabled=False, indent=False )
ext_dreambooth = widgets.Checkbox( value=False, description='Dreambooth', disabled=False, indent=False )

modat_controlnet = widgets.Checkbox( value=False, description='ControlNet 1.1', disabled=False, indent=False )

path_cpkt = widgets.Text( value="", placeholder='Path to main checkpoint file', disabled=False )
dir_cpkt = widgets.Text( value="", placeholder='Path to checkpoints folder', disabled=False )
dir_controlnet = widgets.Text( value="", placeholder='Path to controlnet models folder', disabled=False )
dir_outdir_samples = widgets.Text( value="", placeholder='Save all generated images path', disabled=False )
dir_outdir_grids = widgets.Text( value="", placeholder='Save all generated grids path', disabled=False )

ckpt_15 = widgets.Checkbox( value=False, description='v1.5', disabled=False, indent=False )
ckpt_15inpaint = widgets.Checkbox( value=False, description='v1.5 inpaint', disabled=False, indent=False )
ckpt_instruct_pix2pix = widgets.Checkbox( value=False, description='InstructPix2Pix', disabled=False, indent=False )
ckpt_21 = widgets.Checkbox( value=False, description='v2.1', disabled=False, indent=False )
safet_mdjrnyv4 = widgets.Checkbox( value=False, description='openjourney', disabled=False, indent=False )

def btn_continue_event_handler(btn_object):
    grid.close();
    print('You pressed {}'.format(btn_object.description))
    print('Creating .ini')
    config_object = ConfigParser()
    config_object["CONF"] = {
        "ext_controlnet": ext_controlnet.value,
        "ext_openposeeditor": ext_openposeeditor.value,
        "ext_civitai": ext_civitai.value,
        "ext_huggingface": ext_huggingface.value,
        "ext_webuimodeltoolkit": ext_webuimodeltoolkit.value,
        "ext_ultimateupscale": ext_ultimateupscale.value,
        "ext_additionalnetworks": ext_additionalnetworks.value,
        "ext_dynamicprompts": ext_dynamicprompts.value,
        "ext_datasettageditor": ext_datasettageditor.value,
        "ext_dreambooth": ext_dreambooth.value,
        "path_cpkt": path_cpkt.value,
        "dir_cpkt": dir_cpkt.value,
        "dir_controlnet": dir_controlnet.value,
        "dir_outdir_samples": dir_outdir_samples.value,
        "dir_outdir_grids": dir_outdir_grids.value,
        "ckpt_15": ckpt_15.value,
        "ckpt_15inpaint": ckpt_15inpaint.value,
        "ckpt_instruct_pix2pix": ckpt_instruct_pix2pix.value,
        "ckpt_21": ckpt_21.value,
        "safet_mdjrnyv4": safet_mdjrnyv4.value,
        "modat_controlnet": modat_controlnet.value,
    }
    with open(INI_FILE_LOC, 'w') as conf:
        config_object.write(conf)
    clear_output(); print('[1;32m Settings saved and confirmed! ✓')

# - - - - - - 
btn_continue = widgets.Button(description='Save configuration and continue', button_style='info', layout=Layout(width='auto', height='auto'))
btn_continue.on_click(btn_continue_event_handler)

btn1 = widgets.Button(description='EXTENSIONS', layout=Layout(width='auto', height='auto'))
btn1.style.button_color='White'
btn1.style.font_weight='bold'
btn2 = widgets.Button(description='PATHS', layout=Layout(width='auto', height='auto'))
btn2.style.button_color='White'
btn2.style.font_weight='bold'
btn3 = widgets.Button(description='CHECKPOINTS', layout=Layout(width='auto', height='auto'))
btn3.style.button_color='White'
btn3.style.font_weight='bold'
btn4 = widgets.Button(description='MODELS', layout=Layout(width='auto', height='auto'))
btn4.style.button_color='White'
btn4.style.font_weight='bold'

grid = GridspecLayout(7, 5)

grid[0, 0] = btn1
grid[1, 0] = ext_controlnet
grid[2, 0] = ext_openposeeditor
grid[3, 0] = ext_civitai
grid[4, 0] = ext_huggingface
grid[5, 0] = ext_ultimateupscale

grid[1, 1] = ext_additionalnetworks
grid[2, 1] = ext_dynamicprompts
grid[3, 1] = ext_webuimodeltoolkit
grid[4, 1] = ext_datasettageditor
grid[5, 1] = ext_dreambooth

grid[0, 2] = btn3
grid[1, 2] = ckpt_15
grid[2, 2] = ckpt_15inpaint
grid[3, 2] = ckpt_instruct_pix2pix
grid[4, 2] = ckpt_21
grid[5, 2] = safet_mdjrnyv4

grid[0, 3] = btn4
grid[1, 3] = modat_controlnet

grid[0, 4] = btn2
grid[1, 4] = path_cpkt
grid[2, 4] = dir_cpkt
grid[3, 4] = dir_controlnet
grid[4, 4] = dir_outdir_samples
grid[5, 4] = dir_outdir_grids
grid[6, :] = btn_continue

#display(Markdown('**EXTENSIONS**'))
display(grid)

inip = Path(INI_FILE_LOC)
if inip.is_file():
    # Load values
    config_object = ConfigParser()
    config_object.read(INI_FILE_LOC)
    savedconf = config_object["CONF"]

    ext_controlnet.value = getValue(savedconf, 'ext_controlnet', 'bool')
    ext_openposeeditor.value = getValue(savedconf, 'ext_openposeeditor', 'bool')
    ext_civitai.value = getValue(savedconf, 'ext_civitai', 'bool')
    ext_huggingface.value = getValue(savedconf, 'ext_huggingface', 'bool')
    ext_webuimodeltoolkit.value = getValue(savedconf, 'ext_webuimodeltoolkit', 'bool')
    ext_ultimateupscale.value = getValue(savedconf, 'ext_ultimateupscale', 'bool')
    ext_additionalnetworks.value = getValue(savedconf, 'ext_additionalnetworks', 'bool')
    ext_dynamicprompts.value = getValue(savedconf, 'ext_dynamicprompts', 'bool')
    ext_datasettageditor.value = getValue(savedconf, 'ext_datasettageditor', 'bool')
    ext_dreambooth.value = getValue(savedconf, 'ext_dreambooth', 'bool')
    modat_controlnet.value = getValue(savedconf, 'modat_controlnet', 'bool')
    path_cpkt.value = getValue(savedconf, 'path_cpkt', 'string')
    dir_cpkt.value = getValue(savedconf, 'dir_cpkt', 'string')
    dir_controlnet.value = getValue(savedconf, 'dir_controlnet', 'string')
    dir_outdir_samples.value = getValue(savedconf, 'dir_outdir_samples', 'string')
    dir_outdir_grids.value = getValue(savedconf, 'dir_outdir_grids', 'string')
    ckpt_15.value = getValue(savedconf, 'ckpt_15', 'bool')
    ckpt_15inpaint.value = getValue(savedconf, 'ckpt_15inpaint', 'bool')
    ckpt_instruct_pix2pix.value = getValue(savedconf, 'ckpt_instruct_pix2pix', 'bool')
    ckpt_21.value = getValue(savedconf, 'ckpt_21', 'bool')
    safet_mdjrnyv4.value = getValue(savedconf, 'safet_mdjrnyv4', 'bool')
    # https://tutswiki.com/read-write-config-files-in-python/



 Settings saved and confirmed! ✓


In [ ]:
#@title Install and launch the Automatic1111 WebUi
#@markdown ![Load your config](https://raw.githubusercontent.com/steinhaug/stable-diffusion/main/assets/colab-split-start.png)

skip_install = True #@param {type:"boolean"}
#@markdown When you have installed the system, or for some reason restart the process you can check the above to just launch the installed software.

from IPython.display import clear_output;
%cd /content/
clear_output

if not skip_install:
    print('[1;32mStarting install...')  
    %pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

    #if ext_controlnet.value:
    #    !rm -Rf /content/stable-diffusion-webui/extensions/openpose-editor/
    #    !rm -Rf /content/stable-diffusion-webui/extensions/sd-webui-controlnet/
    #    !git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
    #    !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
        #!git clone https://github.com/fkunn1326/openpose-editor/tree/3c3803ded7b5cd66d93bde904963a6f242a27b70 /content/stable-diffusion-webui/extensions/openpose-editor
        #!git clone https://github.com/Mikubill/sd-webui-controlnet/tree/ba2c8a1a82a254c99cbe507769b14b00ff6907ca /content/stable-diffusion-webui/extensions/sd-webui-controlnet

    if ext_controlnet.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/controlnet/
        !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/controlnet

    if modat_controlnet.value:
    #    !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_openpose.pth
    #    !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_depth.pth
    #    !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_sd15_scribble.pth
        do_download = True
        import re
        if re.search('content/', dir_controlnet.value):
            filepath = Path(dir_controlnet.value + '/control_v11p_sd15_openpose.pth')
            if filepath.is_file():
                do_download = False
                print('[1;32mSkipping Controlnet 1.1 models')
        if do_download:
            !wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth -O /content/stable-diffusion-webui/extensions/controlnet/models/control_v11p_sd15_openpose.pth
            !wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.yaml -O /content/stable-diffusion-webui/extensions/controlnet/models/control_v11p_sd15_openpose.yaml
            !wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth -O /content/stable-diffusion-webui/extensions/controlnet/models/control_v11f1p_sd15_depth.pth
            !wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.yaml -O /content/stable-diffusion-webui/extensions/controlnet/models/control_v11f1p_sd15_depth.yaml
            !wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth -O /content/stable-diffusion-webui/extensions/controlnet/models/control_v11p_sd15_scribble.pth
            !wget https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.yaml -O /content/stable-diffusion-webui/extensions/controlnet/models/control_v11p_sd15_scribble.yaml

    if ext_openposeeditor.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/openpose-editor/
        !git clone https://github.com/fkunn1326/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor

    if ext_additionalnetworks.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/webui-additional-networks/
        !git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/webui-additional-networks

    if ext_dynamicprompts.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/dynamic-prompts/
        !git clone https://github.com/adieyal/sd-dynamic-prompts /content/stable-diffusion-webui/extensions/dynamic-prompts

    if ext_datasettageditor.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/dataset-tag-editor/
        !git clone https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor /content/stable-diffusion-webui/extensions/dataset-tag-editor

    if ext_civitai.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/civitai-browser/
        !rm -Rf /content/stable-diffusion-webui/extensions/webui-images-browser/
        !git clone https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/civitai-browser
        !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/webui-images-browser

    if ext_webuimodeltoolkit.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/webui-model-toolkit/
        !git clone https://github.com/arenatemp/stable-diffusion-webui-model-toolkit /content/stable-diffusion-webui/extensions/webui-model-toolkit

    if ext_ultimateupscale.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/ultimate-upscale/
        !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimate-upscale

    if ext_huggingface.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/webui-huggingface/
        !git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/webui-huggingface
        !pip install huggingface-hub==0.11.1

    if ext_dreambooth.value:
        !rm -Rf /content/stable-diffusion-webui/extensions/dreambooth_extension/
        !git clone https://github.com/d8ahazard/sd_dreambooth_extension /content/stable-diffusion-webui/extensions/dreambooth_extension

    if ckpt_15.value:
        filepath = Path(dir_cpkt.value + '/v1-5-pruned-emaonly.ckpt')
        if filepath.is_file():
            print('[1;32mCheckpoint already downloaded, v1-5-pruned-emaonly.ckpt')
        else:
            filepath = Path(dir_cpkt.value + '/v1-5-pruned.ckpt')
            if filepath.is_file():
                print('[1;32mCheckpoint already downloaded, v1-5-pruned.ckpt')
            else:
                #!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5.ckpt
                !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt
                !wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.vae.pt

    if ckpt_15inpaint.value:
        filepath = Path(dir_cpkt.value + '/v1-5-inpainting.ckpt')
        if filepath.is_file():
            print('[1;32mCheckpoint already downloaded, v1-5-inpainting.ckpt')
        else:
            !wget https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v1-5-inpainting.ckpt

    if ckpt_instruct_pix2pix.value:
        filepath = Path(dir_cpkt.value + '/instruct-pix2pix.ckpt')
        if filepath.is_file():
            print('[1;32mCheckpoint already downloaded, instruct-pix2pix.ckpt')
        else:
            !wget https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/instruct-pix2pix.ckpt

    if safet_mdjrnyv4.value:
        filepath = Path(dir_cpkt.value + '/mdjrny-v4.safetensors')
        if filepath.is_file():
            print('[1;32mCheckpoint already downloaded, mdjrny-v4.safetensors')
        else:
            !wget https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.safetensors -O /content/stable-diffusion-webui/models/Stable-diffusion/mdjrny-v4.safetensors

    if ckpt_21.value:
        filepath = Path(dir_cpkt.value + '/v2-1_768.ckpt')
        if filepath.is_file():
            print('[1;32mCheckpoint already downloaded, v2-1_768.ckpt')
        else:
            !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768.ckpt
    print('[1;32mInstalling complete!')

else:
    print('[1;32mSkipping installers... done!')  

%cd /content/stable-diffusion-webui/

if not skip_install:
    inip = Path('/content/stable-diffusion-webui/config.json')
    if not inip.is_file():
        print('[1;32mCreating WebUI config.json')
        import json
        conf_ini_json = {
            "outdir_samples": dir_outdir_samples.value,
            "outdir_grids": dir_outdir_samples.value,
            "control_net_models_path": modat_controlnet.value,
            "hide_samplers": [
                "LMS",
                "Heun",
                "DPM2",
                "DPM2 a",
                "DPM++ 2S a",
                "DPM++ 2M",
                "DPM++ SDE",
                "DPM fast",
                "DPM adaptive",
                "LMS Karras",
                "PLMS",
                "DPM++ 2S a Karras",
                "DPM2 a Karras",
                "DPM2 Karras"
            ],
            "unload_models_when_training": True,
            "training_xattention_optimizations": True
        }
        with open("config.json", "w") as jsonfile:
            json.dump(conf_ini_json, jsonfile)
    else:
        print('[1;32mSkipping creation of WebUI config.json, already exists')

args = "--share --xformers --enable-insecure-extension-access"

if skip_install:
    args = args + " --skip-install"
if path_cpkt.value != "":
    args = args + " --ckpt=\"" + path_cpkt.value + "\""
if dir_cpkt.value != "":
    args = args + " --ckpt-dir=\"" + dir_cpkt.value + "\""

clear_output
print('[1;32mLaunching WebUI')  
!python launch.py $args


# Different helper cells in case problems

In [ ]:
#@title Error messages said we need upgrade torchvision and torch, fix
!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchtext torchaudio torchdata --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
#@title Error with pycairo and building wheels, fix
! sudo apt install libcairo2-dev pkg-config python3-dev
! pip install pycairo

In [ ]:
#@title Download a file
%cd /content/drive/MyDrive/models
!wget  --content-disposition https://civitai.com/api/download/models/65269